This gets the DataFrame as backend for Visualization. For now, this completes everything for July.

## Getting and saving the cleaned twitter data
* We get the necessary columns for the twitter data from the  .json files by running the code on the cluster and saving the dataframes as parquet.
* The code to get the data is in cleaned_twitter.py

In [1]:
import os
import sys

spark_path = os.environ["SPARK_PATH"]
os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

In [2]:
import pandas as pd
import os
from tqdm import tqdm
from pyspark import SparkConf, SparkContext
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import time

conf = SparkConf().setAppName("ADA-gcl")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
#sqlContext.setConf("spark.sql.parquet.compression.codec", "snappy")


## Load the paqueret data

## DataFrames
* df1: num_followings, num_followers, mentions
* df2: gender, geo_state
* df3: language, sentiment, location, main, date, 

In [3]:
path_user_info = 'file:///Users/tbfang/Documents/EPFL/ADA/ADA-Project/twitter_data/twitter_user_info_jul.parquet'
df1 = sqlContext.read.parquet(path_user_info)

path_gender = 'file:///Users/tbfang/Documents/EPFL/ADA/ADA-Project/twitter_data/twitter_gender_jul.parquet'
df2 = sqlContext.read.parquet(path_gender)

path_sentiment = 'file:///Users/tbfang/Documents/EPFL/ADA/ADA-Project/twitter_data/twitter_cleaned_jul.parquet'
df3 = sqlContext.read.parquet(path_sentiment)

In [4]:
df1 = df1.toPandas()
df2 = df2.toPandas()
df3 = df3.toPandas()

In [5]:
print(df1.shape[0], df2.shape[0], df3.shape[0])

795026 795026 795026


In [6]:
orig_df = df1.merge(df2,on='id').merge(df3,on='id')

In [81]:
orig_df.head()

,id,num_followers,num_following,mentions,canton,gender,language,sentiment,location,main,date
0,1467359435565400044,66,89.0,[Urs_Buhler],None,FEMALE,en,NEGATIVE,Svizzera,@Urs_Buhler and hear then you deserve so much ...,2016-07-01T07:46:46Z
1,1467365796334500043,66,89.0,[Urs_Buhler],None,FEMALE,en,POSITIVE,Svizzera,"@Urs_Buhler can not be bought, and we must alw...",2016-07-01T09:35:26Z
2,1467359036377100186,66,89.0,[Urs_Buhler],None,FEMALE,en,POSITIVE,Svizzera,"@Urs_Buhler In that, there's always someone wh...",2016-07-01T07:41:32Z
3,1467366116249400147,66,89.0,[Urs_Buhler],None,FEMALE,en,POSITIVE,Svizzera,@Urs_Buhler Am I wrong? So bravo Urs. Let me g...,2016-07-01T09:37:31Z
4,1467365778307300193,66,89.0,[Urs_Buhler],None,FEMALE,en,POSITIVE,Svizzera,@Urs_Buhler I consider every person in this wo...,2016-07-01T09:34:21Z


In [7]:
df = df1.merge(df2,on='id').merge(df3,on='id')

In [8]:
code_mapping = pd.read_csv('twitter_data/canton_code.csv').ix[:,1:3]

In [9]:
code_mapping.columns = ['canton_code','canton']

In [84]:
code_mapping.head()

,canton_code,canton
0,AG,Aargau
1,AI,Appenzell Inner Rhoden
2,AR,Appenzell Outer Rhoden
3,BS,Basel-City
4,BL,Basle-Country


In [10]:
df = code_mapping.merge(df, on='canton', how='inner')

In [11]:
df.count()

canton_code      357881
canton           357881
id               357881
num_followers    357881
num_following    357815
mentions         119627
gender           357868
language         357881
sentiment        316497
location         343200
main             357881
date             357881
dtype: int64

## Analyzing DF
* 28/80, ~35% haveinfo about chatters
* 48/80, ~60% have cantons > 50%, pretty good!
* 70/80, ~87.5% have sentiment

### DataFrame Transformations
* DONE mentions -> number
* DONE parse date into month and time of day (6am to 6pm) (6pm to 6am) is night

In [12]:
df.mentions.fillna("",inplace=True)

### converting date

In [13]:
def month(iso_time):
    time_struct = time.strptime(iso_time, "%Y-%m-%dT%H:%M:%SZ")
    return time_struct.tm_mon

In [14]:
def time_of_day(iso_time):
    time_struct = time.strptime(iso_time, "%Y-%m-%dT%H:%M:%SZ")
    hour = time_struct.tm_hour
    if 6 <= hour < 18:
        return "Day"
    else:
        return "Night"

In [15]:
df

,canton_code,canton,id,num_followers,num_following,mentions,gender,language,sentiment,location,main,date
0,AG,Aargau,1467354438402300035,34,97.0,,MALE,U,None,Baden,#schweiz #switzerland #suisse #swiss #svizzera...,2016-07-01T06:23:45Z
1,AG,Aargau,1467398079407500174,1634,1588.0,[simonshirley72],UNKNOWN,en,POSITIVE,Switzerland,@simonshirley72 go #storm didn't see that scor...,2016-07-01T18:31:53Z
2,AG,Aargau,1467384385377100009,728,254.0,,FEMALE,en,POSITIVE,Switzerland,In vet's with Aethelflaed. The good thing abou...,2016-07-01T14:43:11Z
3,AG,Aargau,1467403635318000029,1634,1588.0,[hannahmchugh_],UNKNOWN,en,POSITIVE,Switzerland,@hannahmchugh_ Not a good day at all.,2016-07-01T20:05:03Z
4,AG,Aargau,1467380876526700035,1634,1588.0,"[callyowl, malcolm_fox2, babygibbo]",UNKNOWN,U,None,Switzerland,@callyowl @malcolm_fox2 @babygibbo,2016-07-01T13:45:23Z
5,AG,Aargau,1467392773200400142,1634,1588.0,[ClareNewton],UNKNOWN,en,NEUTRAL,Switzerland,@ClareNewton Do you remember him in a league o...,2016-07-01T17:02:08Z
6,AG,Aargau,1467399513351000032,1634,1588.0,[Flecko_85],UNKNOWN,en,NEGATIVE,Switzerland,@Flecko_85 Somebody gob savage please,2016-07-01T18:56:53Z
7,AG,Aargau,1467402859112200147,1634,1588.0,[garethwalker13],UNKNOWN,en,NEUTRAL,Switzerland,@garethwalker13 Absolute class with that pass,2016-07-01T19:51:18Z
8,AG,Aargau,1467400753120200170,1634,1588.0,[chrisbraiths],UNKNOWN,en,NEUTRAL,Switzerland,@chrisbraiths think in gang bang movie,2016-07-01T19:17:56Z
9,AG,Aargau,1467359010331800094,1634,1588.0,,UNKNOWN,en,POSITIVE,Switzerland,Happy Freddo Friday everybodypic.twitter.com/S...,2016-07-01T07:40:12Z


In [16]:
df['num_mentions'] = df.mentions.apply(len)
df['time_of_day'] = df.date.apply(time_of_day)
df['month'] = df.date.apply(month)

In [17]:
df.head()

,canton_code,canton,id,num_followers,num_following,mentions,gender,language,sentiment,location,main,date,num_mentions,time_of_day,month
0,AG,Aargau,1467354438402300035,34,97.0,,MALE,U,None,Baden,#schweiz #switzerland #suisse #swiss #svizzera...,2016-07-01T06:23:45Z,0,Day,7
1,AG,Aargau,1467398079407500174,1634,1588.0,[simonshirley72],UNKNOWN,en,POSITIVE,Switzerland,@simonshirley72 go #storm didn't see that scor...,2016-07-01T18:31:53Z,1,Night,7
2,AG,Aargau,1467384385377100009,728,254.0,,FEMALE,en,POSITIVE,Switzerland,In vet's with Aethelflaed. The good thing abou...,2016-07-01T14:43:11Z,0,Day,7
3,AG,Aargau,1467403635318000029,1634,1588.0,[hannahmchugh_],UNKNOWN,en,POSITIVE,Switzerland,@hannahmchugh_ Not a good day at all.,2016-07-01T20:05:03Z,1,Night,7
4,AG,Aargau,1467380876526700035,1634,1588.0,"[callyowl, malcolm_fox2, babygibbo]",UNKNOWN,U,None,Switzerland,@callyowl @malcolm_fox2 @babygibbo,2016-07-01T13:45:23Z,3,Day,7


## Tweet Level

In [18]:
df_author = df.drop(['mentions','location','main','date','num_mentions'],axis=1)
df.drop(['mentions','location','main','date','num_followers','num_following'],axis=1,inplace=True)
df.dropna(inplace=True)
# keeps 48/80 = 60% of data
# keeps only gender is known

In [19]:
df

,canton_code,canton,id,gender,language,sentiment,num_mentions,time_of_day,month
1,AG,Aargau,1467398079407500174,UNKNOWN,en,POSITIVE,1,Night,7
2,AG,Aargau,1467384385377100009,FEMALE,en,POSITIVE,0,Day,7
3,AG,Aargau,1467403635318000029,UNKNOWN,en,POSITIVE,1,Night,7
5,AG,Aargau,1467392773200400142,UNKNOWN,en,NEUTRAL,1,Day,7
6,AG,Aargau,1467399513351000032,UNKNOWN,en,NEGATIVE,1,Night,7
7,AG,Aargau,1467402859112200147,UNKNOWN,en,NEUTRAL,1,Night,7
8,AG,Aargau,1467400753120200170,UNKNOWN,en,NEUTRAL,1,Night,7
9,AG,Aargau,1467359010331800094,UNKNOWN,en,POSITIVE,0,Day,7
10,AG,Aargau,1467399850120200039,UNKNOWN,en,POSITIVE,1,Night,7
11,AG,Aargau,1467407941091700175,UNKNOWN,en,POSITIVE,1,Night,7


## Cleaning up Cantons

Currently, there are 170 "cantons" from geo_state, that needs to be cleaned...

In [20]:
len(pd.DataFrame(df['canton'].unique(),columns=['Canton']))

23

### Groupby Columns:

In [21]:
dimensions = ['canton','gender']
# ['canton','gender','language','time_of_day','month']

## Handling Authors

Getting distinct authors only, I assume if they have the same # of followers, following and gender they are the same person.

In [22]:
df_author.shape[0]

357881

In [23]:
df_author.drop_duplicates(['num_followers', 'num_following', 'gender'], inplace=True) # only the authors

Need a notion of sample size, has to be significant. Need to remove outliers for num_followers

In [24]:
df_author = df_author.groupby(dimensions, as_index=False).mean()

In [25]:
sentiment_map = {'NEUTRAL': 0, 'POSITIVE':1, 'NEGATIVE':-1}
# df['sentiment_val'] = 
df['sentiment'] = df.sentiment.map(sentiment_map)

In [26]:
df_user = df.groupby(dimensions, as_index=False).mean()

In [27]:
df_user

,canton,gender,sentiment,num_mentions,month
0,Aargau,FEMALE,0.127791,0.608930,7.0
1,Aargau,MALE,0.133448,0.540092,7.0
2,Aargau,UNKNOWN,0.116435,0.265360,7.0
3,Basel-City,FEMALE,0.117866,0.348015,7.0
4,Basel-City,MALE,0.144045,0.480843,7.0
5,Basel-City,UNKNOWN,0.110907,0.199252,7.0
6,Bern,FEMALE,0.180866,0.551327,7.0
7,Bern,MALE,0.179935,0.591930,7.0
8,Bern,UNKNOWN,0.114347,0.414804,7.0
9,Fribourg,FEMALE,0.138062,0.594047,7.0


In [28]:
df.groupby(dimensions, as_index=False).count().head()

,canton,gender,canton_code,id,language,sentiment,num_mentions,time_of_day,month
0,Aargau,FEMALE,1299,1299,1299,1299,1299,1299,1299
1,Aargau,MALE,1746,1746,1746,1746,1746,1746,1746
2,Aargau,UNKNOWN,11019,11019,11019,11019,11019,11019,11019
3,Basel-City,FEMALE,1612,1612,1612,1612,1612,1612,1612
4,Basel-City,MALE,3367,3367,3367,3367,3367,3367,3367


In [29]:
df_final = df_user.merge(df_author,on=dimensions)

In [30]:
df_final.sentiment.describe()

count    69.000000
mean      0.122887
std       0.071263
min      -0.016667
25%       0.080153
50%       0.114347
75%       0.166276
max       0.365123
Name: sentiment, dtype: float64

In [31]:
df_final.shape[0]

69

### Evaluating the goodness of the conditions
We want to make sure there is enough in each category. We want to minimize the # of 0 sentiments.

In [32]:
df_final[df_final['sentiment'] == 0].shape[0] # sentiment = 0 means 

0

In [85]:
df_final

,canton,gender,sentiment,num_mentions,month_x,num_followers,num_following,month_y
0,Aargau,FEMALE,0.127791,0.608930,7.0,1040.484277,447.761006,7.0
1,Aargau,MALE,0.133448,0.540092,7.0,1107.967972,711.362989,7.0
2,Aargau,UNKNOWN,0.116435,0.265360,7.0,2027.768010,502.990220,7.0
3,Basel-City,FEMALE,0.117866,0.348015,7.0,785.612903,628.225806,7.0
4,Basel-City,MALE,0.144045,0.480843,7.0,2080.744493,549.889868,7.0
5,Basel-City,UNKNOWN,0.110907,0.199252,7.0,2925.112561,524.760196,7.0
6,Bern,FEMALE,0.180866,0.551327,7.0,1036.480899,626.211712,7.0
7,Bern,MALE,0.179935,0.591930,7.0,2487.847619,489.279661,7.0
8,Bern,UNKNOWN,0.114347,0.414804,7.0,4271.895341,760.864956,7.0
9,Fribourg,FEMALE,0.138062,0.594047,7.0,962.918750,476.450000,7.0


## First DF

* gender
* canton
* sentiment value

In [34]:
df_simple = df_final[['canton','gender','sentiment']]

In [35]:
code_mapping

,canton_code,canton
0,AG,Aargau
1,AI,Appenzell Inner Rhoden
2,AR,Appenzell Outer Rhoden
3,BS,Basel-City
4,BL,Basle-Country
5,BE,Bern
6,FR,Fribourg
7,GE,Geneva
8,GL,Glarus
9,GR,Grisons


DataFrame processing:
* add other cantons to the dataframe
* set gender, sentiment to 0
* add canton abbreviation
* change unknown to "all" (not even necessary)

Then go to "Discretize Crime Rates" section... (just copy and paste the code... :D)

In [36]:
df_simple = df_simple.merge(code_mapping,how='inner')

In [37]:
# Manual add... 
r = {'Appenzell Inner Rhoden': 'FEMALE', 'Appenzell Inner Rhoden': 'MALE', 'Appenzell Inner Rhoden': 'UNKNOWN',
     'Appenzell Outer Rhoden': 'FEMALE', 'Appenzell Outer Rhoden': 'MALE', 'Appenzell Outer Rhoden': 'UNKNOWN',
     'Basle-Country': 'FEMALE', 'Basle-Country': 'MALE', 'Basle-Country': 'UNKNOWN'}

r = {'Appenzell Inner Rhoden': ['FEMALE','MALE','UNKNOWN'],
     'Appenzell Outer Rhoden': ['FEMALE','MALE','UNKNOWN'],
     'Basle-Country': ['FEMALE','MALE','UNKNOWN']}

# pd.DataFrame(r.items(), columns= ['canton','gender'])
manual = pd.DataFrame.from_dict(r,orient='index')

In [38]:
pd.DataFrame(manual.stack())

0
Appenzell Inner Rhoden 0   FEMALE
                       1     MALE
                       2  UNKNOWN
Basle-Country          0   FEMALE
                       1     MALE
                       2  UNKNOWN
Appenzell Outer Rhoden 0   FEMALE
                       1     MALE
                       2  UNKNOWN

In [39]:
manual = pd.DataFrame(manual.stack()).reset_index()
manual.columns = ['canton','level','gender']

In [40]:
manual = manual[['canton','gender']]

In [41]:
manual['sentiment'] = 0
manual = manual.merge(code_mapping)

In [42]:
manual.head()

,canton,gender,sentiment,canton_code
0,Appenzell Inner Rhoden,FEMALE,0,AI
1,Appenzell Inner Rhoden,MALE,0,AI
2,Appenzell Inner Rhoden,UNKNOWN,0,AI
3,Basle-Country,FEMALE,0,BL
4,Basle-Country,MALE,0,BL


In [43]:
df_simple.head()

,canton,gender,sentiment,canton_code
0,Aargau,FEMALE,0.127791,AG
1,Aargau,MALE,0.133448,AG
2,Aargau,UNKNOWN,0.116435,AG
3,Basel-City,FEMALE,0.117866,BS
4,Basel-City,MALE,0.144045,BS


In [44]:
df_simple = df_simple.append(manual).reset_index().drop(['index'],axis=1)

In [45]:
df_simple.head()

,canton,gender,sentiment,canton_code
0,Aargau,FEMALE,0.127791,AG
1,Aargau,MALE,0.133448,AG
2,Aargau,UNKNOWN,0.116435,AG
3,Basel-City,FEMALE,0.117866,BS
4,Basel-City,MALE,0.144045,BS


In [46]:
df_simple.to_csv('gender_viz.csv')

In [67]:
df_simple = df_simple.groupby('canton_code').mean().reset_index()

## Building the Swiss Sentiment Map

In [98]:
import folium
import json
import branca

In [107]:
# Build the Scale with the Legend
Legend = folium.colormap.linear.YlGn.scale( 0, 0.3 )
Legend = Legend.to_step(10)
Legend.caption = 'Happiness of Swiss Cantons'
Legend

In [108]:
# Load geojson data
geo_json_data = json.load(open(r'insta_data/ch-cantons.geojson.json'))
#
canton_dict = df_simple.set_index('canton_code')['sentiment']

In [109]:
swiss_map = folium.Map(location=[47, 8],tiles='cartodbpositron', zoom_start=8)

g = folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': Legend(canton_dict[feature['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '1',
        'fillOpacity': 0.7,
    }
)

g.add_to(swiss_map)

Legend.add_to(swiss_map)

swiss_map

## Final DF

* canton, gender, language, time of day, month are things we want to filter by
* sentiment, num mentions, num followers, and num_following are all averages for those specified categories. For num_followers and num_followings, it only averages the unique tweet authors. Sentiment and num mentions are averages over all tweets in those filters.

* will have big DataFrame for July to October. Good enough to do visualization...

Need to do:
* map location to Google Maps to cantons for January to June.

Spark/cluster work:
* January gender needs to be fixed

## creating the Canton Map

It's not possible to do location to canton mapping... So we need to map location to cantons directly. We lose too many locations. We need to use Google Maps to automatically map this... This is for January to June. 

In [ ]:
orig_df.groupby('location').count().sort('id', ascending=False).head(100)

In [ ]:
canton_mapping = orig_df[['canton','location']].dropna().drop_duplicates()

In [ ]:
canton_mapping.groupby('canton').count().sort('location',ascending=False)

In [ ]:
pd.DataFrame(canton_mapping.canton.unique(),columns=['Canton'])

In [ ]:
code_mapping = pd.read_csv('twitter_data/canton_code.csv').ix[:,1:3]

In [ ]:
code_mapping.head()
code_mapping.columns = ['canton_code','canton']

In [ ]:
code_mapping.head()

In [ ]:
code_mapping.merge(canton_mapping, on='canton', how='inner')

In [ ]:
canton_mapping.head()

In [ ]:
canton_mapping.groupby('location').count().sort('canton',ascending=False)

Creating a Canton to Location Mapping

In [ ]:
swiss_cantons = ['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 
                 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH']

In [ ]:
len(swiss_cantons)